In [103]:
# import dependencies
import pandas as pd
import json
import requests
from sqlalchemy import create_engine
from config import wlm_api
from pandas.io.json import json_normalize
import time

In [3]:
# Walmart api key from config file
apiKey = wlm_api

In [4]:
# api Call to walmart for the product selected 20 max on 1 call
product_api = (f"http://api.walmartlabs.com/v1/items?ids=51800359,986978174,42490113,317677075,545584365,55449983,38676400,189542267,55420123,38317400,275846958,50820749,47090515,39073013,878167&apiKey={apiKey}&lsPublisherId=").format(apiKey)

In [97]:
# request call to APU
response = requests.get(product_api)
productList = response.json()
# convert api call to DataFrame
productList = pd.DataFrame.from_dict(productList['items'], orient='columns', dtype=None)
# Grab data desired and use as column name
productList = productList[['itemId', 'name', 'salePrice', 'clearance', 'stock']]
# display the dataframe
productList

,itemId,name,salePrice,clearance,stock
0,51800359,Mr. Coffee 12 Cup Programmable Black Coffee Maker,24.94,False,Available
1,986978174,"Power 6-Quart AirFryer Oven Plus, Black",129.00,False,Available
2,42490113,Haier 2.7 Cu Ft Single Door Compact Refrigerat...,109.00,False,Available
3,317677075,"SAMSUNG 82"" Class 4K Ultra HD (2160P) HDR Smar...",5997.99,False,Available
4,545584365,DJI Mavic 2 Pro,1323.13,False,Available
5,55449983,"Nintendo Switch Console with Gray Joy-Con, HAC...",299.00,False,Available
6,38676400,Yodelling Pickle Musical Toy,12.29,False,Available
7,189542267,"VIZIO SmartCast 38"" 3.1 Channel Soundbar Syste...",138.00,False,Available
8,55420123,MEGA BOXING GLOVES,5.92,False,Not available
9,38317400,"Mainstays 8-Cube Bookcase, White or Espresso",79.80,False,Available


In [182]:
# create a list of the items for api call loop
itemId = [51800359,986978174,42490113,317677075,545584365,55449983,38676400,189542267,55420123,38317400,
              275846958,50820749,47090515,39073013,878167]
items = []
# review_product_api
for item in itemId:
    url = 'http://api.walmartlabs.com/v1/reviews/' + str(item) + '?apiKey=' + str(apiKey) + '&lsPublisherId=&format=json'
    response = requests.get(url)
    prodReview = response.json()
    time.sleep(2)
#         json.dumps(response)
    items.append(prodReview)

In [330]:
# convert api call to DataFrame
prodReview = pd.DataFrame.from_dict(items, orient='columns', dtype=None)
# Grab data desired and use as column name
prodReview = prodReview[['itemId','reviewStatistics']]
# for kwy in prodReview:
#     print(kwy)

In [331]:
reviewStats = prodReview['reviewStatistics'].apply(pd.Series)

In [333]:
prodRatingOver = pd.concat([prodReview, reviewStats], axis=1)
prodRatingOver = prodRatingOver[['itemId', 'averageOverallRating', 'overallRatingRange', 'totalReviewCount']]
prodRatingOver

,itemId,averageOverallRating,overallRatingRange,totalReviewCount
0,51800359,3.83,5,888
1,986978174,3.81,5,188
2,42490113,4.33,5,420
3,317677075,4.29,5,17
4,545584365,4.5,5,2
5,55449983,4.68,5,182
6,38676400,5.0,5,1
7,189542267,4.33,5,178
8,55420123,2.5,5,10
9,38317400,4.28,5,223


In [334]:
mergedDF = pd.merge(productList, prodRatingOver, on='itemId')
mergedDF

,itemId,name,salePrice,clearance,stock,averageOverallRating,overallRatingRange,totalReviewCount
0,51800359,Mr. Coffee 12 Cup Programmable Black Coffee Maker,24.94,False,Available,3.83,5,888
1,986978174,"Power 6-Quart AirFryer Oven Plus, Black",129.00,False,Available,3.81,5,188
2,42490113,Haier 2.7 Cu Ft Single Door Compact Refrigerat...,109.00,False,Available,4.33,5,420
3,317677075,"SAMSUNG 82"" Class 4K Ultra HD (2160P) HDR Smar...",5997.99,False,Available,4.29,5,17
4,545584365,DJI Mavic 2 Pro,1323.13,False,Available,4.5,5,2
5,55449983,"Nintendo Switch Console with Gray Joy-Con, HAC...",299.00,False,Available,4.68,5,182
6,38676400,Yodelling Pickle Musical Toy,12.29,False,Available,5.0,5,1
7,189542267,"VIZIO SmartCast 38"" 3.1 Channel Soundbar Syste...",138.00,False,Available,4.33,5,178
8,55420123,MEGA BOXING GLOVES,5.92,False,Not available,2.5,5,10
9,38317400,"Mainstays 8-Cube Bookcase, White or Espresso",79.80,False,Available,4.28,5,223
